In [1]:
import sys
sys.path.append('../')
# from experiments import utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plot.utils import bar_update_fig, line_update_fig
import pypolo2

width = 1000
height = 800
markersize = 10
linewidth = 4
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
linetype = {'Low':'dot','Medium':'dashdot','High':'longdash'}
# 创建一个包含线型的示例数组
line_dash = ['solid', 'dot', 'dash', 'dashdot', 'longdash']
colors_dict = ['#C74223', '#3D9182', '#245C8F']

In [2]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "EffectOrientedGreedySpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_meanpm = []
sche_step = 0
adaptive_step = 0
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_meanpm = []
    for i in range(3):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        time_meanpm = []
        for time_point in range(48):
            seed_meanpm = []
            for seed in seeds:
                pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
                pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
                data = pypolo2.experiments.utilities.readpkl(pkldir)
                pm = data['truth_env']
                # 获取特定时间点的效果数据
                seed_meanpm.append(np.mean(pm[time_point]))
            time_meanpm.append(np.mean(seed_meanpm))
        variable_meanpm.append(time_meanpm)
    strategy_meanpm.append(variable_meanpm)
strategy_meanpm = np.array(strategy_meanpm)


In [3]:
width = 1000
height = 620
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 1
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 48
for i in range(Time_deadline):
    labels.append(int(i*10))

xtitle = 'time(min)'
ytitle = 'Average pm2.5 concentration (μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][0][0:Time_deadline], name='SmartSpr+3 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][1][0:Time_deadline], name='SmartSpr+5 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][2][0:Time_deadline], name='SmartSpr+7 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['High'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][0][0:Time_deadline], name='MaximumCoverage+3 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[0],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][1][0:Time_deadline], name='MaximumCoverage+5 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[0],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][2][0:Time_deadline], name='MaximumCoverage+7 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[0],dash=linetype['High'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[3][0][0:Time_deadline], name='Non-Sprinkle', marker=dict(size=markersize, symbol=markersymbol['DDRL']),line=dict(color=colors_dict[1], width=linewidth),))

fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=30),tickfont=dict(size=30))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=30),tickfont=dict(size=30))
fig.update_layout(
    yaxis=dict(range=[0, 70]),
    legend=dict(font=dict(size=22), x=0.5, y=1.4)) # name size
fig.show()
filename = './outputs/png/line_timeAveragePollution-teamsize357-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_timeAveragePollution-teamsize357-linetype.png


# Average pm2.5 concentration for different team size

In [13]:
#区域平均污染物浓度
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_4","teamsize_5","teamsize_6","teamsize_7"]]
seeds = [0,3,7,11,15,18,20,32,42,50,60,66,70]
team_sizes = [3,4,5,6,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25concentration = []
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25concentration = []
    for i in range(5):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        seed_pm25concentration = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            time_pm25concentration = []
            for j in range(total_step):
                if j > 24:
                    pm25concentration = np.mean(data['truth_env'][j])
                    time_pm25concentration.append(pm25concentration)
            seed_pm25concentration.append(np.mean(time_pm25concentration))
        variable_pm25concentration.append(np.mean(seed_pm25concentration))
    strategy_pm25concentration.append(variable_pm25concentration)
strategy_pm25concentration = np.array(strategy_pm25concentration)
strategy_pm25concentration[-1,:] = np.mean(strategy_pm25concentration[-1,:])
print(strategy_pm25concentration)


[[35.8079768  31.51729042 30.19393804 24.02974966 24.08973282]
 [48.63786615 45.86228313 39.35888149 37.94853008 35.31758814]
 [62.16149997 62.16149997 62.16149997 62.16149997 62.16149997]]


In [17]:
width = 1000
height = 600
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 15
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 5
for i in range(Time_deadline):
    labels.append(int(i + 3))

xtitle = 'Sprinkle Truck Number'
ytitle = 'Average pm2.5 concerntion(μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_pm25concentration[0][0:Time_deadline], name='SmartSpr', marker=dict(size=markersize, symbol=markersymbol['FH']),line=dict(color=colors_dict[2], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_pm25concentration[1][0:Time_deadline], name='MaximumCoverage', marker=dict(size=markersize, symbol=markersymbol['HEUR']),line=dict(color=colors_dict[0], width=linewidth),))


fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, 
                title_font=dict(size=25),
                tickfont=dict(size=20),
                tickmode='array',
                tickvals=labels,
                ticktext=[str(label) for label in labels],)
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, 60]),
    legend=dict(font=dict(size=32), x=0.5, y=1.15)) # name size
fig.show()

filename = './outputs/png/line_AverageConcention-teamszize34567-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_AverageConcention-teamszize34567-linetype.png


# Average suppression of pm2.5 concentration for diff team size

In [18]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_4","teamsize_5","teamsize_6","teamsize_7"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,4,5,6,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25suppression = []
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25suppression = []
    for i in range(5):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        seed_pm25suppression = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            effect = np.sum(data['spray_effect'])/48/400
            seed_pm25suppression.append(effect)
        variable_pm25suppression.append(np.mean(seed_pm25suppression))
    strategy_pm25suppression.append(variable_pm25suppression)
strategy_pm25suppression = np.array(strategy_pm25suppression)
print(strategy_pm25suppression)

[[ 1.59316220e+00  1.74091271e+00  1.95580597e+00  1.98742308e+00
   2.05179502e+00]
 [ 8.35927209e-01  9.84986893e-01  1.13681589e+00  1.20766360e+00
   1.46069469e+00]
 [-6.83201464e-05  1.67357421e-04  1.86714469e-05  1.72393930e-05
  -9.97748662e-05]]


In [20]:
width = 1000
height = 600
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 15
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 5
for i in range(Time_deadline):
    labels.append(int(i*1 + 3))

xtitle = 'Sprinkle Truck Number'
ytitle = 'Average Suppression of pm2.5 (μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_pm25suppression[0][0:Time_deadline], name='SmartSpr', marker=dict(size=markersize, symbol=markersymbol['FH']),line=dict(color=colors_dict[2], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_pm25suppression[1][0:Time_deadline], name='MaximumCoverage', marker=dict(size=markersize, symbol=markersymbol['HEUR']),line=dict(color=colors_dict[0], width=linewidth),))

fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, 
                title_font=dict(size=25),
                tickfont=dict(size=20),
                tickmode='array',
                tickvals=labels,
                ticktext=[str(label) for label in labels],)
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, 2.5]),
    legend=dict(font=dict(size=32), x=0.5, y=1.15)) # name size
fig.show()
filename = './outputs/png/line_AverageSuppression-teamsize34567-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_AverageSuppression-teamsize34567-linetype.png


# Average pm2.5 concentration for different source number

In [2]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_meanpm = []
sche_step = 0
adaptive_step = 0
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_meanpm = []
    for i in range(3):
        comparing_variable = comparing_variables[1][i]
        team_size = 5
        sourcenum = sourcenums[i]
        time_meanpm = []
        for time_point in range(48):
            seed_meanpm = []
            for seed in seeds:
                pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
                pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
                data = pypolo2.experiments.utilities.readpkl(pkldir)
                pm = data['truth_env']
                # 获取特定时间点的效果数据
                seed_meanpm.append(np.mean(pm[time_point]))
            time_meanpm.append(np.mean(seed_meanpm))
        variable_meanpm.append(time_meanpm)
    strategy_meanpm.append(variable_meanpm)
strategy_meanpm = np.array(strategy_meanpm)

In [11]:
width = 1000
height = 620
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 1
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 48
for i in range(Time_deadline):
    labels.append(int(i*10))

xtitle = 'time(min)'
ytitle = 'Average pm2.5 concentration (μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][0][0:Time_deadline], name='SmartSpr+2 Source', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][1][0:Time_deadline], name='SmartSpr+4 Source', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][2][0:Time_deadline], name='SmartSpr+6 Source', mode='lines', line=dict(color=colors_dict[2],dash=linetype['High'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[1][0][0:Time_deadline], name='MaximumCoverage+2 Source', mode='lines', line=dict(color=colors_dict[0],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[1][1][0:Time_deadline], name='MaximumCoverage+4 Source', mode='lines', line=dict(color=colors_dict[0],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[1][2][0:Time_deadline], name='MaximumCoverage+6 Source', mode='lines', line=dict(color=colors_dict[0],dash=linetype['High'], width=linewidth),))
# fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][0][0:Time_deadline], name='Non-Sprinkle', marker=dict(size=markersize, symbol=markersymbol['DDRL']),line=dict(color=colors_dict[0], width=linewidth),))

fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=30),tickfont=dict(size=30))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=30),tickfont=dict(size=30))
fig.update_layout(
    yaxis=dict(range=[0, 100]),
    legend=dict(font=dict(size=22), x=0.5, y=1.35)) # name size
fig.show()
filename = './outputs/png/line_timeAveragePollution-source246-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_timeAveragePollution-source246-linetype.png


# Average suppression for 2468 source

In [6]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["numsource_2","numsource_4","numsource_6","numsource_8"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,4,5,6,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25suppression = []
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25suppression = []
    for i in range(4):
        comparing_variable = comparing_variables[0][i]
        team_size = 5
        sourcenum = sourcenums[i]
        seed_pm25suppression = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            effect = np.sum(data['spray_effect'])/48/400
            seed_pm25suppression.append(effect)
        variable_pm25suppression.append(np.mean(seed_pm25suppression))
    strategy_pm25suppression.append(variable_pm25suppression)
strategy_pm25suppression = np.array(strategy_pm25suppression)
print(strategy_pm25suppression)


[[ 1.34512182e+00  3.12238010e+00  4.47962192e+00  5.49473746e+00]
 [ 7.36390587e-01  2.06939874e+00  3.18291553e+00  3.95072158e+00]
 [-9.65541990e-05 -9.48863411e-05 -3.47639144e-04 -2.69435105e-04]]


In [8]:
width = 1000
height = 600
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 15
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 4
for i in range(Time_deadline):
    labels.append(int(i*2 + 2))

xtitle = 'Source Number'
ytitle = 'Average Suppression of pm2.5 (μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_pm25suppression[0][0:Time_deadline], name='SmartSpr', marker=dict(size=markersize, symbol=markersymbol['FH']),line=dict(color=colors_dict[2], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_pm25suppression[1][0:Time_deadline], name='MaximumCoverage', marker=dict(size=markersize, symbol=markersymbol['HEUR']),line=dict(color=colors_dict[0], width=linewidth),))

fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, 7]),
    legend=dict(font=dict(size=32), x=0.5, y=1.15)) # name size
fig.show()
filename = './outputs/png/line_AverageSuppression-source2468-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_AverageSuppression-source2468-linetype.png


In [9]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["numsource_2","numsource_4","numsource_6","numsource_8"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,4,5,6,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25concentration = []

for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25concentration = []
    for i in range(4):
        comparing_variable = comparing_variables[0][i]
        team_size = 5
        sourcenum = sourcenums[i]
        seed_pm25concentration = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            time_pm25concentration = []
            for j in range(total_step):
                if j > 24:
                    pm25concentration = np.mean(data['truth_env'][j])
                    time_pm25concentration.append(pm25concentration)
            seed_pm25concentration.append(np.mean(time_pm25concentration))
        variable_pm25concentration.append(np.mean(seed_pm25concentration))
    strategy_pm25concentration.append(variable_pm25concentration)
strategy_pm25concentration = np.array(strategy_pm25concentration)

In [10]:
width = 1000
height = 600
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 15
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 4
for i in range(Time_deadline):
    labels.append(int(i*2 + 2))

xtitle = 'Source Number'
ytitle = 'Average pm2.5 concerntion(μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_pm25concentration[0][0:Time_deadline], name='SmartSpr', marker=dict(size=markersize, symbol=markersymbol['FH']),line=dict(color=colors_dict[2], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_pm25concentration[1][0:Time_deadline], name='MaximumCoverage', marker=dict(size=markersize, symbol=markersymbol['HEUR']),line=dict(color=colors_dict[0], width=linewidth),))

fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, 100]),
    legend=dict(font=dict(size=32), x=0.5, y=1.15)) # name size
fig.show()

filename = './outputs/png/line_AverageConcention-source2468-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_AverageConcention-source2468-linetype.png
